# CONSTANTS

In [1]:
DATASET_SOURCE = "datasets/extracted/dataset-cleaned.csv"

# TRAINING CONFIGS

EPOCHS = 5
BATCH_SIZE = 6
LEARNING_RATE = 1e-5
SEED = 7984

MODEL_SAVE_PATH = "models/sucidality"
MODEL_CHECKPOINT_PATH = "models/sucidality-checkpoint"
MODEL_LOGGING_PATH = "models/sucidality-checkpoint/logs"


# Setup

## Install dependencies

In [4]:
!pip install transformers transformers[torch] datasets numpy pandas scikit-learn 

/bin/bash: /home/ubuntu/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## Import all dependencies

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

## Load cleaned dataset

In [ ]:
ds = pd.read_csv(DATASET_SOURCE, lineterminator='\n')
ds = ds[['cleaned', 'class']]
ds = ds.rename(columns={'cleaned': 'text', 'class': 'label'})
ds = ds.dropna()

ds

## Split dataset for training, evaluation and testing.

In [9]:
train, temp = train_test_split(ds, random_state=SEED, test_size=0.2, stratify=ds['label'])
val, test = train_test_split(temp, random_state=SEED, test_size=0.5, stratify=temp['label'])

train, temp, val, test

(                                                     text  label
 156594  consider buy gun want die live state get gun e...      1
 116160  game thingie explain happen game lose billion ...      0
 128769                                 feel sad feel well      0
 1202    not understand college coach not run ball dump...      0
 240691  want know bear pain want normal die tired way ...      1
 ...                                                   ...    ...
 142070  jewish new year eve year tough wish good new y...      0
 215373  vegan teacher got ban celebrate no soggy tit s...      0
 85      wish relate read post help feel like write hop...      1
 179933     get commitment issue try fix tell somebody get      0
 153753             spineless moronic fucking matter tired      1
 
 [209613 rows x 2 columns],
                                                      text  label
 169025  fight boyfriend stay parent house not want sta...      1
 138009  mon pass away past summer good friend

# Setup Model & tokenize the input texts.

In [10]:
device = torch.device("cuda")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("gooohjy/suicidal-electra")

In [12]:
def dataset_conversion(train, test, val):
  """Converts pandas dataframe to Dataset."""

  train.reset_index(drop=True, inplace=True)
  test.reset_index(drop=True, inplace=True)
  val.reset_index(drop=True, inplace=True)

  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)
  val_dataset = Dataset.from_pandas(val)

  return DatasetDict({"train": train_dataset, "test": test_dataset, "val": val_dataset})

raw_datasets = dataset_conversion(train, test, val)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 209613
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 26202
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 26202
    })
})

In [13]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)
    
tokenized_datasets = raw_datasets.map(tokenize_function, batched=False)
tokenized_datasets

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26202/26202 [00:07<00:00, 3534.24 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 209613
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 26202
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 26202
    })
})

# Training

## Load the model & trainer methods

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("gooohjy/suicidal-electra", num_labels=2)

In [34]:
# Define custom metrics for computation

def compute_metrics(eval_pred):
    metric_acc = load_metric("accuracy")
    metric_rec = load_metric("recall")
    metric_pre = load_metric("precision")
    metric_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    recall = metric_rec.compute(predictions=predictions, references=labels)["recall"]
    precision = metric_pre.compute(predictions=predictions, references=labels)["precision"]
    f1 = metric_f1.compute(predictions=predictions, references=labels)["f1"]

    return {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1': f1}

In [91]:
# Define model and training parameters

def get_trainer(datasets, epochs=EPOCHS):

    training_args = TrainingArguments(
        output_dir=MODEL_CHECKPOINT_PATH,
        overwrite_output_dir = True,
        learning_rate=LEARNING_RATE,
        num_train_epochs=epochs,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        seed=SEED,
        # evaluation_strategy="epoch",
        logging_dir=MODEL_LOGGING_PATH,
        save_strategy="steps",
        save_steps=1500
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets['train'],
        eval_dataset=datasets['val'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

## Train a sample

In [92]:
SAMPLE_SIZE = 20

sample = {
    'train': tokenized_datasets['train'].shuffle(seed=SEED).select(range(SAMPLE_SIZE)),
    'test': tokenized_datasets['test'].shuffle(seed=SEED).select(range(SAMPLE_SIZE)),
    'val': tokenized_datasets['val'].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
}

sample

{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 20
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 20
 }),
 'val': Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 20
 })}

In [94]:
trainer = get_trainer(sample)
trainer.train(resume_from_checkpoint=True)

Step,Training Loss


TrainOutput(global_step=20, training_loss=1.2119610346417175e-07, metrics={'train_runtime': 3.1428, 'train_samples_per_second': 31.819, 'train_steps_per_second': 6.364, 'total_flos': 26311105536000.0, 'train_loss': 1.2119610346417175e-07, 'epoch': 5.0})

In [95]:
result = trainer.evaluate()

pd.DataFrame(data=result, index=['SAMPLE TEST RESULTS']).transpose()

,SAMPLE TEST RESULTS
eval_loss,0.842932
eval_accuracy,0.950000
eval_recall,0.909091
eval_precision,1.000000
eval_f1,0.952381
eval_runtime,0.478900
eval_samples_per_second,41.760000
eval_steps_per_second,8.352000
epoch,5.000000


## Train model with the complete dataset

In [ ]:
trainer = get_trainer(tokenized_datasets)
trainer.train()

/home/ubuntu/miniconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.213500
1000,0.266700
1500,0.354100
2000,0.342100
2500,0.326700
3000,0.331000
3500,0.329500
4000,0.339800
4500,0.315700
5000,0.305100


In [ ]:
result = trainer.evaluate()
pd.DataFrame(data=result, index=['SAMPLE TEST RESULTS']).transpose()

## Save model

In [ ]:
trainer.save_model(MODEL_SAVE_PATH)